<a href="https://colab.research.google.com/github/michelup/TEST_SDA/blob/main/Project2_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalace

In [1]:
pip install ydata-profiling


In [2]:
pip install lazypredict


In [3]:
import kagglehub
import os
import pandas as pd

from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from lazypredict.Supervised import LazyClassifier

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

from sklearn import tree
import graphviz
import seaborn as sns

# Data

In [4]:
# Download latest version
slozka = kagglehub.dataset_download("ahsan81/hotel-reservations-classification-dataset")

In [5]:
soubor = os.listdir(slozka)[0]

In [6]:
data = pd.read_csv(slozka + os.sep + soubor, index_col="Booking_ID")

In [7]:
data

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
Booking_ID,,,,,,,,,,,,,,,,,,
INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INN36271,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,167.80,1,Not_Canceled
INN36272,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,90.95,2,Canceled
INN36273,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,98.39,2,Not_Canceled


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36275 entries, INN00001 to INN36275
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          36275 non-null  int64  
 1   no_of_children                        36275 non-null  int64  
 2   no_of_weekend_nights                  36275 non-null  int64  
 3   no_of_week_nights                     36275 non-null  int64  
 4   type_of_meal_plan                     36275 non-null  object 
 5   required_car_parking_space            36275 non-null  int64  
 6   room_type_reserved                    36275 non-null  object 
 7   lead_time                             36275 non-null  int64  
 8   arrival_year                          36275 non-null  int64  
 9   arrival_month                         36275 non-null  int64  
 10  arrival_date                          36275 non-null  int64  
 11  market_seg

In [9]:
data.describe( )

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
count,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00,36275.00
mean,1.84,0.11,0.81,2.20,0.03,85.23,2017.82,7.42,15.60,0.03,0.02,0.15,103.42,0.62
std,0.52,0.40,0.87,1.41,0.17,85.93,0.38,3.07,8.74,0.16,0.37,1.75,35.09,0.79
min,0.00,0.00,0.00,0.00,0.00,0.00,2017.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00
25%,2.00,0.00,0.00,1.00,0.00,17.00,2018.00,5.00,8.00,0.00,0.00,0.00,80.30,0.00
50%,2.00,0.00,1.00,2.00,0.00,57.00,2018.00,8.00,16.00,0.00,0.00,0.00,99.45,0.00
75%,2.00,0.00,2.00,3.00,0.00,126.00,2018.00,10.00,23.00,0.00,0.00,0.00,120.00,1.00
max,4.00,10.00,7.00,17.00,1.00,443.00,2018.00,12.00,31.00,1.00,13.00,58.00,540.00,5.00


In [10]:
#POZN  MARKET SEGMENT ROZDELIT DO 3 a zbytek OTHER

# Define the column you want to modify
column_to_modify = 'market_segment_type'

# Define the list of specific values to group into 'Other'
values_to_group = ['Corporate', 'Aviation', 'Complementary']

# Check the current value counts before grouping (optional but recommended)
print(f"Original value counts for '{column_to_modify}':")
print(data[column_to_modify].value_counts())

# Use the replace method to change the specified values to 'Other'
# Using .loc to ensure proper assignment and avoid
data.loc[:, column_to_modify] = data[column_to_modify].replace(values_to_group, 'Other')

# Check the value counts after grouping to verify the change
print(f"\nValue counts for '{column_to_modify}' after grouping:")
print(data[column_to_modify].value_counts())

# Display the head of the DataFrame to see the changes (optional)
column_to_show = 'market_segment_type'
print(f"\nUnique values in the '{column_to_show}' column:")
print(data[column_to_show].unique())

Original value counts for 'market_segment_type':
market_segment_type
Online           23214
Offline          10528
Corporate         2017
Complementary      391
Aviation           125
Name: count, dtype: int64

Value counts for 'market_segment_type' after grouping:
market_segment_type
Online     23214
Offline    10528
Other       2533
Name: count, dtype: int64

Unique values in the 'market_segment_type' column:
['Offline' 'Online' 'Other']


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36275 entries, INN00001 to INN36275
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          36275 non-null  int64  
 1   no_of_children                        36275 non-null  int64  
 2   no_of_weekend_nights                  36275 non-null  int64  
 3   no_of_week_nights                     36275 non-null  int64  
 4   type_of_meal_plan                     36275 non-null  object 
 5   required_car_parking_space            36275 non-null  int64  
 6   room_type_reserved                    36275 non-null  object 
 7   lead_time                             36275 non-null  int64  
 8   arrival_year                          36275 non-null  int64  
 9   arrival_month                         36275 non-null  int64  
 10  arrival_date                          36275 non-null  int64  
 11  market_seg

# EDA

In [12]:
df=data.reset_index()

In [13]:
# NAHRAZENI 0 HODNOT  prumerem z odpovidajici  kategorie  pokoju

avg_prices = df.groupby('room_type_reserved')['avg_price_per_room'].mean()
print(avg_prices)  # Zobrazíme průměrné ceny pro každou kategorii
df['avg_price_per_room'] = df.apply(
    lambda row: avg_prices[row['room_type_reserved']] if row['avg_price_per_room'] == 0 else row['avg_price_per_room'], axis=1
)

print(df.head())  # Zobrazíme upravený DataFrame


room_type_reserved
Room_Type 1    95.92
Room_Type 2    87.85
Room_Type 3    73.68
Room_Type 4   125.29
Room_Type 5   123.73
Room_Type 6   182.21
Room_Type 7   155.20
Name: avg_price_per_room, dtype: float64
  Booking_ID  no_of_adults  no_of_children  no_of_weekend_nights  \
0   INN00001             2               0                     1   
1   INN00002             2               0                     2   
2   INN00003             1               0                     2   
3   INN00004             2               0                     0   
4   INN00005             2               0                     1   

   no_of_week_nights type_of_meal_plan  required_car_parking_space  \
0                  2       Meal Plan 1                           0   
1                  3      Not Selected                           0   
2                  1       Meal Plan 1                           0   
3                  2       Meal Plan 1                           0   
4                  1      Not Sele

In [14]:
# ONE HOT enc.
from sklearn.preprocessing import OneHotEncoder

# Inicializace encoderu
encoder = OneHotEncoder(sparse_output=False)

# Transformace dat
encoded_array = encoder.fit_transform(df[['type_of_meal_plan','room_type_reserved','market_segment_type']])

# Vytvoření nového DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out())

#prevod z float na integer
encoded_df = encoded_df.astype(int)

In [15]:
#df

In [16]:
#novy data frame
encoded_df

#neslo nam spojit tabulky, dokud jsme neresetovali index

df_encoded=encoded_df.reset_index(drop=True)
df_encoded

,type_of_meal_plan_Meal Plan 1,type_of_meal_plan_Meal Plan 2,type_of_meal_plan_Meal Plan 3,type_of_meal_plan_Not Selected,room_type_reserved_Room_Type 1,room_type_reserved_Room_Type 2,room_type_reserved_Room_Type 3,room_type_reserved_Room_Type 4,room_type_reserved_Room_Type 5,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Offline,market_segment_type_Online,market_segment_type_Other
0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
1,0,0,0,1,1,0,0,0,0,0,0,0,1,0
2,1,0,0,0,1,0,0,0,0,0,0,0,1,0
3,1,0,0,0,1,0,0,0,0,0,0,0,1,0
4,0,0,0,1,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,1,0,0,0,0,0,0,1,0,0,0,0,1,0
36271,1,0,0,0,1,0,0,0,0,0,0,0,1,0
36272,1,0,0,0,1,0,0,0,0,0,0,0,1,0
36273,0,0,0,1,1,0,0,0,0,0,0,0,1,0


In [17]:
#reset indexu

df_reset = df.reset_index(drop=True)

#df_reset

In [18]:
#pripojeni puvodniho datasetu k one-hot datasetu
df=df_reset.join(encoded_df)

#odstraneni Booking_ID sloupce a puvodniho "type of meal"
df.drop(columns=['type_of_meal_plan','Booking_ID','room_type_reserved','market_segment_type'], inplace=True)

df

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,...,room_type_reserved_Room_Type 1,room_type_reserved_Room_Type 2,room_type_reserved_Room_Type 3,room_type_reserved_Room_Type 4,room_type_reserved_Room_Type 5,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Offline,market_segment_type_Online,market_segment_type_Other
0,2,0,1,2,0,224,2017,10,2,0,...,1,0,0,0,0,0,0,1,0,0
1,2,0,2,3,0,5,2018,11,6,0,...,1,0,0,0,0,0,0,0,1,0
2,1,0,2,1,0,1,2018,2,28,0,...,1,0,0,0,0,0,0,0,1,0
3,2,0,0,2,0,211,2018,5,20,0,...,1,0,0,0,0,0,0,0,1,0
4,2,0,1,1,0,48,2018,4,11,0,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,3,0,2,6,0,85,2018,8,3,0,...,0,0,0,1,0,0,0,0,1,0
36271,2,0,1,3,0,228,2018,10,17,0,...,1,0,0,0,0,0,0,0,1,0
36272,2,0,2,6,0,148,2018,7,1,0,...,1,0,0,0,0,0,0,0,1,0
36273,2,0,0,3,0,63,2018,4,21,0,...,1,0,0,0,0,0,0,0,1,0


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          36275 non-null  int64  
 1   no_of_children                        36275 non-null  int64  
 2   no_of_weekend_nights                  36275 non-null  int64  
 3   no_of_week_nights                     36275 non-null  int64  
 4   required_car_parking_space            36275 non-null  int64  
 5   lead_time                             36275 non-null  int64  
 6   arrival_year                          36275 non-null  int64  
 7   arrival_month                         36275 non-null  int64  
 8   arrival_date                          36275 non-null  int64  
 9   repeated_guest                        36275 non-null  int64  
 10  no_of_previous_cancellations          36275 non-null  int64  
 11  no_of_previous_

In [20]:
#prevod BOOKING STATUS pomoci LABEL encoderu
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['booking_status_encoded'] = encoder.fit_transform(df['booking_status'])


In [21]:
# smazani puvodniho sloupce BOOKING STATUS po labelingu

df.drop(columns=['booking_status'], inplace=True)

In [22]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36275 entries, 0 to 36274
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          36275 non-null  int64  
 1   no_of_children                        36275 non-null  int64  
 2   no_of_weekend_nights                  36275 non-null  int64  
 3   no_of_week_nights                     36275 non-null  int64  
 4   required_car_parking_space            36275 non-null  int64  
 5   lead_time                             36275 non-null  int64  
 6   arrival_year                          36275 non-null  int64  
 7   arrival_month                         36275 non-null  int64  
 8   arrival_date                          36275 non-null  int64  
 9   repeated_guest                        36275 non-null  int64  
 10  no_of_previous_cancellations          36275 non-null  int64  
 11  no_of_previous_

In [23]:
#zjisteni minima - mozne vylepseni , zkontrolovat outliers , 0.50$ za pokoj indikuje mozou chybu

df[['avg_price_per_room']].min()


,0
avg_price_per_room,0.50


In [24]:


pocet_nul = (df['avg_price_per_room'] == 0).sum()
pocet_nul

np.int64(0)

In [25]:
#vyvazenost datasetu -  potreba vybalancovat

data["booking_status"].value_counts()

#not cancelled = 1
#cancelled = 0

,count
booking_status,
Not_Canceled,24390
Canceled,11885


In [26]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

#Dle HEATMAPy zavisi koreluje booking status nejvice s

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,type_of_meal_plan_Meal Plan 1,type_of_meal_plan_Meal Plan 2,type_of_meal_plan_Meal Plan 3,type_of_meal_plan_Not Selected,room_type_reserved_Room_Type 1,room_type_reserved_Room_Type 2,room_type_reserved_Room_Type 3,room_type_reserved_Room_Type 4,room_type_reserved_Room_Type 5,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Offline,market_segment_type_Online,market_segment_type_Other,booking_status_encoded
no_of_adults,1.000000,-0.019787,0.103316,0.105622,0.011429,0.097287,0.076719,0.021841,0.026338,-0.192277,-0.047426,-0.119166,0.283256,0.189401,-0.029246,0.017804,0.003509,0.020643,-0.267437,-0.071375,-0.007326,0.299972,-0.017421,0.047788,0.051244,-0.082993,0.242919,-0.309731,-0.086920
no_of_children,-0.019787,1.000000,0.029478,0.024398,0.034244,-0.047091,0.045983,-0.003076,0.025482,-0.036348,-0.016390,-0.021189,0.362275,0.124486,0.083539,-0.025921,-0.003070,-0.079789,-0.258013,0.167241,-0.003633,-0.074656,0.005709,0.650342,0.113705,-0.133710,0.154620,-0.053103,-0.033078
no_of_weekend_nights,0.103316,0.029478,1.000000,0.179577,-0.031111,0.046595,0.055357,-0.009894,0.027304,-0.067107,-0.020690,-0.026312,-0.034572,0.060593,0.044326,-0.047693,-0.008236,-0.014086,-0.065915,0.018976,-0.003818,0.063118,-0.005147,0.013736,-0.005334,-0.058901,0.116152,-0.113874,-0.061563
no_of_week_nights,0.105622,0.024398,0.179577,1.000000,-0.048784,0.149650,0.032672,0.037376,-0.009305,-0.099764,-0.030080,-0.049344,-0.012654,0.045994,0.088059,-0.041908,-0.005029,-0.072001,-0.104278,0.011087,-0.002012,0.106852,-0.000491,0.014763,-0.002457,-0.010714,0.080895,-0.133284,-0.092996
required_car_parking_space,0.011429,0.034244,-0.031111,-0.048784,1.000000,-0.066445,0.015684,-0.015505,-0.000037,0.110909,0.027106,0.063810,0.073865,0.087922,0.025045,-0.031738,0.011452,-0.004545,-0.035314,0.014605,-0.002484,0.017200,-0.004131,0.035641,0.014747,-0.102429,0.049616,0.088963,0.086185
lead_time,0.097287,-0.047091,0.046595,0.149650,-0.066445,1.000000,0.143439,0.136814,0.006479,-0.135984,-0.045723,-0.078137,-0.103199,-0.101645,-0.054360,0.229702,-0.010444,-0.123427,0.095383,0.025624,-0.002532,-0.083503,-0.025642,-0.046615,-0.037052,0.280103,-0.153570,-0.209583,-0.438538
arrival_year,0.076719,0.045983,0.055357,0.032672,0.015684,0.143439,1.000000,-0.339689,0.018849,-0.018175,0.003919,0.026422,0.162269,0.053205,0.018087,-0.169844,-0.000625,0.118352,-0.103895,-0.011937,-0.009012,0.104697,0.023266,0.018498,0.016766,-0.163854,0.182180,-0.051330,-0.179529
arrival_month,0.021841,-0.003076,-0.009894,0.037376,-0.015505,0.136814,-0.339689,1.000000,-0.042780,0.000341,-0.038615,-0.010718,0.066455,0.110552,-0.020708,0.030233,-0.010800,0.000507,0.013507,-0.019311,0.012306,-0.012835,0.002081,0.006624,0.005736,0.031059,-0.018776,-0.019947,0.011233
arrival_date,0.026338,0.025482,0.027304,-0.009305,-0.000037,0.006479,0.018849,-0.042780,1.000000,-0.015949,-0.012542,-0.001498,0.014663,0.018352,-0.014977,0.024123,0.002959,-0.001861,-0.034562,0.014523,-0.000722,0.024601,0.000141,0.018675,0.003816,-0.014699,0.014280,-0.000720,-0.010629
repeated_guest,-0.192277,-0.036348,-0.067107,-0.099764,0.110909,-0.135984,-0.018175,0.000341,-0.015949,1.000000,0.390810,0.539156,-0.117422,-0.011820,0.072809,-0.045902,0.012953,-0.050816,0.036289,-0.013696,-0.002254,-0.041289,0.020904,-0.015997,0.044892,-0.069139,-0.181371,0.464737,0.107287


In [27]:
# rozdělení na cílovou proměnnou a featury
y = df["booking_status_encoded"]
X = df.drop(columns="booking_status_encoded")

In [28]:
df

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,...,room_type_reserved_Room_Type 2,room_type_reserved_Room_Type 3,room_type_reserved_Room_Type 4,room_type_reserved_Room_Type 5,room_type_reserved_Room_Type 6,room_type_reserved_Room_Type 7,market_segment_type_Offline,market_segment_type_Online,market_segment_type_Other,booking_status_encoded
0,2,0,1,2,0,224,2017,10,2,0,...,0,0,0,0,0,0,1,0,0,1
1,2,0,2,3,0,5,2018,11,6,0,...,0,0,0,0,0,0,0,1,0,1
2,1,0,2,1,0,1,2018,2,28,0,...,0,0,0,0,0,0,0,1,0,0
3,2,0,0,2,0,211,2018,5,20,0,...,0,0,0,0,0,0,0,1,0,0
4,2,0,1,1,0,48,2018,4,11,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,3,0,2,6,0,85,2018,8,3,0,...,0,0,1,0,0,0,0,1,0,1
36271,2,0,1,3,0,228,2018,10,17,0,...,0,0,0,0,0,0,0,1,0,0
36272,2,0,2,6,0,148,2018,7,1,0,...,0,0,0,0,0,0,0,1,0,1
36273,2,0,0,3,0,63,2018,4,21,0,...,0,0,0,0,0,0,0,1,0,0


In [29]:
# skalovani dat

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

In [31]:
# from imblearn.under_sampling import RandomUnderSampler

# Načtení datasetu
# X = X_train
# y = y_train


# Použití undersamplingu - vynecha 50% vzorku z vetsinove tridy
# rus = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
# X_resampled, y_resampled = rus.fit_resample(X, y)


# print(f"Původní velikost: {X.shape}, Po undersamplingu: {X_resampled.shape}")

In [34]:
#not cancelled = 1
#cancelled = 0



not_cancelled = X_train[X_train["booking_status_encoded"] == 1].sample(11200)
cancelled = X_train[X_train["booking_status_encoded"] == 0]
pd.concat([nula, jedna])



# Check the value counts before downsampling (optional)
print("Value counts in y_train before downsampling:")
print(y_train.value_counts())

# Find the indices where y_train is 1
indices_to_downsample = y_train[y_train == 1].index

# Calculate the number of samples to remove (50%)
num_samples_to_remove = int(len(indices_to_downsample) * 0.5)

# Randomly select the indices to remove
indices_to_drop = y_train[y_train == 1].sample(n=num_samples_to_remove, random_state=42).index

# Remove the select
indices from X_train and y_train
y_train = y_train.drop(indices_to_drop)

# Check the value counts after downsampling to verify the change (optional)
print("\nValue counts in y_train after downsampling:")
print(y_train.value_counts())

KeyError: 'booking_status_encoded'

In [ ]:
X_train=X_resampled
y_train=y_resampled

#  KNN model

In [ ]:
from sklearn.metrics import recall_score, balanced_accuracy_score
df_vysledky = {}

In [ ]:
# hodnoty parametrů, kde budeme hledat optimum
parameters_knn = {'n_neighbors':[3, 5, 7, 9, 11],
                  'weights':['uniform', 'distance'],
                  'metric':['manhattan', "cosine", "euclidean"]}
# inicializace modelu - použijeme všechny defaultní parametry - žádné nastavování
knn = KNeighborsClassifier()

In [ ]:
# inicializace optimizátoru gridsearch
gs = GridSearchCV(knn, parameters_knn, verbose=1)
# trénink kombinací a hledání nejlepšího nastavení hyperparametrů
gs.fit(X_train, y_train)

In [ ]:
# výsledky optimalizace
# nejlepší kombinace hyperparametrů
print(gs.best_params_)
# do knn uložíme nejlepší model z optimalizace
knn = gs.best_estimator_

In [ ]:
# udělej odhad na testovacích feautrách
y_pred_knn = knn.predict(X_test)

In [ ]:
# tiskneme výsledky
print(classification_report(y_test, y_pred_knn))
cm = confusion_matrix(y_test, y_pred_knn, labels=knn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=knn.classes_)
disp.plot()

In [ ]:
# vyhodnocení predikcí na trénovacími daty - pozor, z těch se přímo učil
y_pred_train_knn = knn.predict(X_train)
# tiskneme výsledky
print(classification_report(y_train, y_pred_train_knn))
cm = confusion_matrix(y_train, y_pred_train_knn, labels=knn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=knn.classes_)
disp.plot()

In [ ]:


df_vysledky["knn"] = [recall_score(y_test, y_pred_knn, average="weighted"), balanced_accuracy_score(y_test, y_pred_knn)]

# Decison tree model


In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# hodnoty parametrů, kde budeme hledat optimum
parameters_dt = {'max_depth':[3, 5, 7],
                 'criterion':['entropy', 'gini', "log_loss"],
                 'max_features':[None, "sqrt", "log2"]}
# inicializace modelu - použijeme všechny defaultní parametry - žádné nastavování
dt = DecisionTreeClassifier()

In [ ]:
# inicializace optimizátoru gridsearch
gs = GridSearchCV(dt, parameters_dt, verbose=1)
# trénink kombinací a hledání nejlepšího nastavení hyperparametrů
gs.fit(X_train, y_train)

In [ ]:
# výsledky optimalizace
# nejlepší kombinace hyperparametrů
print(gs.best_params_)
# do knn uložíme nejlepší model z optimalizace
dt = gs.best_estimator_

In [ ]:
y_pred_dt = dt.predict(X_test)

In [ ]:
# tiskneme výsledky
print(classification_report(y_test, y_pred_dt))
cm = confusion_matrix(y_test, y_pred_dt, labels=knn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=knn.classes_)
disp.plot()

In [ ]:
#pro srovnani - predikce pro trénovací data

y_pred_train_dt = dt.predict(X_train)

print(classification_report(y_train, y_pred_train_dt))
cm = confusion_matrix(y_train, y_pred_train_dt, labels=knn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=knn.classes_)
disp.plot()

In [ ]:
# Export tree to DOT format
dot_data = tree.export_graphviz(dt,
                                feature_names=dt.feature_names_in_,
                                class_names=dt.classes_.astype(str))

# Convert to graph and save as PNG -> kvůli zoomování, neztrtíme rozlišení
graph = graphviz.Source(dot_data)
graph.render("decision_tree_reservation", format="png")

In [ ]:
df_vysledky["dt"] = [recall_score(y_test, y_pred_dt, average="weighted"), balanced_accuracy_score(y_test, y_pred_dt)]

# Logistic regression model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# hodnoty parametrů, kde budeme hledat optimum
parameters_lr = {'penalty':[None, "l2"],
                 'C':[0.1, 1, 10],
                 'class_weight':[None, "balanced"],
                 'solver':['lbfgs', "liblinear", "saga"]}
# inicializace modelu - použijeme všechny defaultní parametry - žádné nastavování
lr = LogisticRegression()

In [ ]:
# inicializace optimizátoru gridsearch
gs = GridSearchCV(lr, parameters_lr, verbose=1)
# trénink kombinací a hledání nejlepšího nastavení hyperparametrů
gs.fit(X_train, y_train)

In [ ]:
# výsledky optimalizace
# nejlepší kombinace hyperparametrů
print(gs.best_params_)
# do knn uložíme nejlepší model z optimalizace
lr = gs.best_estimator_

In [ ]:
y_pred_lr = lr.predict(X_test)

In [ ]:
# tiskneme výsledky
print(classification_report(y_test, y_pred_lr))
cm = confusion_matrix(y_test, y_pred_lr, labels=knn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=knn.classes_)
disp.plot()

In [ ]:
df_vysledky["lr"] = [recall_score(y_test, y_pred_lr, average="weighted"), balanced_accuracy_score(y_test, y_pred_lr)]

# Random Forest model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ShuffleSplit

In [ ]:
# hodnoty parametrů, kde budeme hledat optimum
parameters_rf = {
    'n_estimators': [10, 50, 100],  # Počet stromů v lese
    'max_depth': [None, 3, 10],  # Maximální hloubka stromu
    'min_samples_split': [2, 5, 10],  # Minimální počet vzorků potřebných k rozdělení uzlu
    'max_features': ['sqrt', None],  # Počet proměnných zvažovaných při dělení
    'bootstrap': [True, False]  # Použití bootstrapu
}

# inicializace modelu - použijeme všechny defaultní parametry - žádné nastavování
rf = RandomForestClassifier()

In [ ]:
# inicializace optimizátoru gridsearch
gs = GridSearchCV(rf, parameters_rf, verbose=1, cv=ShuffleSplit(test_size=0.20, n_splits=1, random_state=0))
# trénink kombinací a hledání nejlepšího nastavení hyperparametrů
gs.fit(X_train, y_train)

In [ ]:
# výsledky optimalizace
# nejlepší kombinace hyperparametrů
print(gs.best_params_)
# do knn uložíme nejlepší model z optimalizace
rf = gs.best_estimator_

In [ ]:
y_pred_rf = rf.predict(X_test)

In [ ]:
# tiskneme výsledky
print(classification_report(y_test, y_pred_rf))
cm = confusion_matrix(y_test, y_pred_rf, labels=knn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=knn.classes_)
disp.plot()

In [ ]:
df_vysledky["rf"] = [recall_score(y_test, y_pred_rf, average="weighted"), balanced_accuracy_score(y_test, y_pred_rf)]

# AdaBoost model

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# hodnoty parametrů, kde budeme hledat optimum
parameters_ab = {
    'n_estimators': [10, 50, 100],  # Počet slabých klasifikátorů
    'learning_rate': [0.01, 0.1, 1]  # Rychlost učení
}

# inicializace modelu - použijeme všechny defaultní parametry - žádné nastavování
ab = AdaBoostClassifier()

In [ ]:
# inicializace optimizátoru gridsearch
gs = GridSearchCV(ab, parameters_ab, verbose=1)
# trénink kombinací a hledání nejlepšího nastavení hyperparametrů
gs.fit(X_train, y_train)

In [ ]:
# výsledky optimalizace
# nejlepší kombinace hyperparametrů
print(gs.best_params_)
# do knn uložíme nejlepší model z optimalizace
ab = gs.best_estimator_

In [ ]:
y_pred_ab = ab.predict(X_test)

In [ ]:
# tiskneme výsledky
print(classification_report(y_test, y_pred_ab))
cm = confusion_matrix(y_test, y_pred_ab, labels=knn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=knn.classes_)
disp.plot()

In [ ]:
df_vysledky["ab"] = [recall_score(y_test, y_pred_ab, average="weighted"), balanced_accuracy_score(y_test, y_pred_ab)]

# Support vector model

In [ ]:
from sklearn.svm import SVC

In [ ]:
# hodnoty parametrů, kde budeme hledat optimum
parameters_svc = {
    'C': [1, 10],  # Regularizační parametr
    'kernel': ['linear', 'rbf'],  # Typ jádra
    'gamma': ['scale', 'auto', 0.1]  # Koeficient jádra (pro rbf/poly)
}

# inicializace modelu - použijeme vto šechny defaultní parametry - žádné nastavování
svc = SVC()

svc.fit(X_train,y_train)

In [ ]:
# GRID search jsme vynechali z duvodu vycerpani systemovych zdroju

# inicializace optimizátoru gridsearch
#   gs = GridSearchCV(svc, parameters_svc, verbose=1)
# trénink kombinací a hledání nejlepšího nastavení hyperparametrů
#    gs.fit(X_train, y_train)

In [ ]:
# výsledky optimalizace
# nejlepší kombinace hyperparametrů
#     print(gs.best_params_)
# do knn uložíme nejlepší model z optimalizace
#   svc = gs.best_estimator_

In [ ]:
y_pred_svc = svc.predict(X_test)

In [ ]:
# tiskneme výsledky
print(classification_report(y_test, y_pred_svc))
cm = confusion_matrix(y_test, y_pred_svc, labels=knn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=knn.classes_)
disp.plot()

In [ ]:
df_vysledky["svc"] = [recall_score(y_test, y_pred_svc, average="weighted"), balanced_accuracy_score(y_test, y_pred_svc)]

# Vysledky

In [ ]:
df_vysledky

In [ ]:
df = pd.DataFrame.from_dict(
    df_vysledky,
    orient='index',
    columns=['recall', 'balanced_accuracy']
)

melted=pd.melt(df.reset_index(), id_vars='index', var_name='metric')

In [ ]:
melted.set_index(['metric', 'index'])

In [ ]:
sns.barplot(x='metric', y='value', hue='index', data=melted)

# Lazy Predict

In [ ]:
# LAZY PREDICT skoncil s chybou kvuli chybejici RAM  - moc dat

from lazypredict.Supervised import LazyClassifier

In [ ]:
lp = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = lp.fit(X_train, X_test, y_train, y_test)

In [ ]:
models.sort_values("F1 Score", ascending=False)